In [1]:
from load_matlab_elmannet import *
import numpy as np
import gym
import time
import os
from PIL import Image

In [2]:
crop = False

In [3]:
if crop:
    y_min, y_max, x_min, x_max = 25+30, 195-40, 20, 140
    shape_of_single_frame = (1, (y_max-y_min),(x_max-x_min))
    flat_image = 12000
else:
    y_min, y_max, x_min, x_max = 25, 195, 20, 140
    shape_of_single_frame = (1, (y_max-y_min),(x_max-x_min))
    flat_image = 20400

In [4]:
dir_path = "../2-elman_train/models/"
dir_path = "../2-lstm_train/best_models/chuncked_sequence/elman/sigmoide_sigmoide/elman_sigmoide_sigmoide_m35to35_f1to120_H200_epoch10000" + "/"
arr = os.listdir(f'./{dir_path}')
for i in range(len(arr)):
    print(arr[i])

confmat.jpg
painel.png
perf.jpg
zigzagM35to35F1to120Epoch5000H200.mat
_home_ryo_.local_lib_python3.8_site-packages_ipykernel_launcher.py (Ubuntu) 2021-05-10 00-32-30.mp4
_home_ryo_.local_lib_python3.8_site-packages_ipykernel_launcher.py (Ubuntu) 2021-05-10 00-34-40.mp4
_home_ryo_.local_lib_python3.8_site-packages_ipykernel_launcher.py (Ubuntu) 2021-05-10 00-36-17.mp4
_home_ryo_.local_lib_python3.8_site-packages_ipykernel_launcher.py (Ubuntu) 2021-05-10 00-39-27.mp4
_home_ryo_.local_lib_python3.8_site-packages_ipykernel_launcher.py (Ubuntu) 2021-05-10 00-39-55.mp4
_home_ryo_.local_lib_python3.8_site-packages_ipykernel_launcher.py (Ubuntu) 2021-05-10 00-40-21.mp4


In [5]:
model_path = dir_path + "zigzagM35to35F1to120Epoch5000H200.mat"

In [6]:
sleep_time = 0.05

chuncked= True
zigzag = True
is_checkpoint= False
match = 35
data_path = r"../1-generate/data/"

start_frame = 1
end_frame = 120

In [7]:
if zigzag:
    ACTIONS = {
        "right": 2,
        "left": 3,
    }
else:
    ACTIONS = {
            "noop": 0,
            "accelerate": 1,
            "right": 2,
            "left": 3,
            "break": 4,
            "right_break": 5,
            "left_break": 6,
            "right_accelerate": 7,
            "left_accelerate": 8,
    }

In [8]:
ACTIONS_LIST = list(ACTIONS.values())

In [9]:
def load_npz(m):
    path = data_path + "npz/"

    actions = np.load(path + 'actions.npz')
    lifes = np.load(path + 'lifes.npz')
    frames = np.load(path + 'frames.npz')
    rewards = np.load(path + 'rewards.npz')

    arr_actions = actions.f.arr_0
    arr_lifes = lifes.f.arr_0
    arr_frames = frames.f.arr_0
    arr_rewards = rewards.f.arr_0

    print("Successfully loaded NPZ.")

    return arr_actions.shape[0], arr_frames, arr_actions, arr_rewards, arr_lifes

In [10]:
def prepare_action_data(action, ACTIONS_LIST):

    new_action = np.zeros((1, len(ACTIONS_LIST)), dtype=int) 

    new_action[0, ACTIONS_LIST.index(action)] = 1

    return new_action

In [11]:
num_of_frames_arr = []
frames_arr = []
actions_arr = []

In [12]:
def load_npz(data_path, m):
    
    path = data_path + "match_" + str(m) + "/npz/"

    actions = np.load(path + 'actions.npz')
    lifes = np.load(path + 'lifes.npz')
    frames = np.load(path + 'frames.npz')
    rewards = np.load(path + 'rewards.npz')

    arr_actions = actions.f.arr_0
    arr_lifes = lifes.f.arr_0
    arr_frames = frames.f.arr_0
    arr_rewards = rewards.f.arr_0

    print("Successfully loaded NPZ.")

    return arr_actions.shape[0], arr_frames, arr_actions, arr_rewards, arr_lifes

In [13]:
num_of_frames_arr = []
frames_arr = []
actions_arr = []

In [14]:
def prepare_action_data(action, ACTIONS_LIST):

    new_action = np.zeros((1, len(ACTIONS_LIST)), dtype=int) 

    new_action[0, ACTIONS_LIST.index(action)] = 1

    return new_action

In [15]:
num_of_frames, frames, actions, rewards, lifes = load_npz(data_path, match)
frames = frames[start_frame - 1:end_frame]
if crop:
    frames = frames.reshape(end_frame - start_frame + 1, 170, 120)
    frames = frames[:, 30:130, :]

frames = frames.reshape(end_frame - start_frame + 1, flat_image)
    
actions = actions[start_frame - 1:end_frame]
action_one_hot = [prepare_action_data(i, ACTIONS_LIST) for i in actions]
actions = np.array(action_one_hot)
actions = actions.reshape(len(actions), -1)

frames_arr.append(frames)
actions_arr.append(actions)
num_of_frames_arr.append(end_frame - start_frame + 1) 

X_train = np.array(frames_arr)/255
Y_train = np.array(actions_arr)
num_of_frames_arr = np.array(num_of_frames_arr)

Successfully loaded NPZ.


In [16]:
model = LoadElman(model_path, is_checkpoint)

/home/ryo/.local/lib/python3.8/site-packages/scipy/io/matlab/mio.py:226: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


U shape: (200, 20400)
W shape: (200, 200)
V shape: (2, 200)
bh shape: (200,)
bo shape: (2,)


In [17]:
Y_train.shape

(1, 120, 2)

In [18]:
model.reset_prev_s()

print_class = False

for i in range(len(frames)):
    if(print_class):
        print(i+1, np.argmax(Y_train[i]), "->", np.argmax(model.predict_step(frames[i]/255)))
    else:
        print(i+1, Y_train[0][i], "->", model.predict_step(frames[i]/255))

1 [1 0] -> [0.99680235 0.00282265]
2 [1 0] -> [0.99570604 0.00263325]
3 [1 0] -> [0.99671645 0.00368793]
4 [1 0] -> [0.995007   0.00580072]
5 [1 0] -> [0.99752904 0.00283554]
6 [1 0] -> [0.99901786 0.00103565]
7 [1 0] -> [0.99747    0.00164439]
8 [1 0] -> [0.99886922 0.0024819 ]
9 [1 0] -> [9.98954439e-01 9.61404348e-04]
10 [1 0] -> [9.99418502e-01 2.63673120e-04]
11 [1 0] -> [9.99602479e-01 6.73689833e-04]
12 [1 0] -> [9.99509838e-01 9.80436694e-04]
13 [1 0] -> [9.99591990e-01 8.07619484e-04]
14 [1 0] -> [9.99497304e-01 3.51451668e-04]
15 [1 0] -> [9.99369122e-01 4.15781932e-04]
16 [1 0] -> [9.99270513e-01 2.97950706e-04]
17 [1 0] -> [9.99108654e-01 7.29830231e-04]
18 [1 0] -> [9.99590044e-01 3.58497176e-04]
19 [1 0] -> [9.99398812e-01 3.66337337e-04]
20 [1 0] -> [0.99903783 0.00107971]
21 [1 0] -> [9.98387626e-01 8.06609586e-04]
22 [1 0] -> [0.99862621 0.00186372]
23 [1 0] -> [0.99769625 0.00195604]
24 [1 0] -> [9.99481580e-01 7.12126235e-04]
25 [1 0] -> [0.9994248  0.00105616]
26 [1

In [19]:
model.reset_prev_s()
predicted = []
correct = 0
wrong = 0
for i in range( len(frames) ):
    
    if np.argmax(Y_train[0][i]) == np.argmax(model.predict_step(frames[i]/255)):
        correct += 1
    else:
        wrong +=1

In [20]:
print(correct)
print(wrong)

120
0


In [ ]:
model.reset_prev_s()

env = gym.make("Enduro-v0")
frame = env.reset()
reward, action, done, info = 0, 0, False, {'ale.lives': 0}
env.render()

time.sleep(10)

for _ in range(1000):
    time.sleep(sleep_time)
    env.render()
    
    frame = frame[y_min:y_max, x_min:x_max]

    frame = Image.fromarray(frame)
    frame = frame.convert("L")
    
    frame = np.asarray(frame)
    frame = frame.reshape(flat_image,)
    frame = frame/255
    
    action = model.predict_step(frame)
    action = list(ACTIONS.values())[np.argmax(action, axis=0)]
    
    print(action)
    
    frame, reward, done, info = env.step(action)

2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
2
2
3
3
3
3
3
3
3
3
3
3
3
3
2
2
2
2
3
3
3
3
3


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/ryo/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-21-9eee094d1377>", line 23, in <module>
    action = model.predict_step(frame)
  File "/mnt/c/Users/alece/Desktop/UnB/tcc/rnn_enduro/3-load_model/load_matlab_elmannet.py", line 36, in predict_step
    mulu = np.dot(self.U, new_input)
  File "<__array_function__ internals>", line 5, in dot
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ryo/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ryo/.local/

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/ryo/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-21-9eee094d1377>", line 23, in <module>
    action = model.predict_step(frame)
  File "/mnt/c/Users/alece/Desktop/UnB/tcc/rnn_enduro/3-load_model/load_matlab_elmannet.py", line 36, in predict_step
    mulu = np.dot(self.U, new_input)
  File "<__array_function__ internals>", line 5, in dot
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ryo/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ryo/.local/